## P300 Demo
* We will work with a dataset where subjects observed images of target and non-target objects while having their neural activity recorded with scalp EEG
* We will create and fit a model that predicts which category of object they saw based on their observed brain activity
* This takes the form of a binary classification problem. 
* The model that we will use is known as a "convolutional" network, due to it transforming the input data in a manner analogous to filtering in linear systems

### Loading the data
* The data is contained in a Matlab data file
* We use the scipy package to conveniently bring it into the Python workspace

In [2]:
filename = '../data/P300.mat' # the data file
import scipy.io as sio # the library that we use to load Matlab data into Python
data = sio.loadmat(filename)

### Inspect the data
* The data has been partitioned into a training and test set
* Features are shaped (example, electrode, time samples)
* Labels are shaped (example, 2), where the 2 refers to the two classes (target vs non-target)

In [3]:
x_train = data['X_train']
x_test = data['X_test']
y_train = data['Y_train']
y_test = data['Y_test']

print("The shape of the training features is " + str(x_train.shape))
print("The shape of the training labels is " + str(y_train.shape))
print("The shape of the test features is " + str(x_test.shape))
print("The shape of the test labels is " + str(y_test.shape))

The shape of the training features is (1058, 64, 128)
The shape of the training labels is (1058, 2)
The shape of the test features is (530, 64, 128)
The shape of the test labels is (530, 2)


### Visualizing a sample trial
* Looking at the data is always recommended
* Here the data takes the form of space-time matrix, that we plot as overlaid waveforms

In [ ]:
import numpy as np # Python's array library
import matplotlib.pyplot as plt # a popular library for plotting in Python
trial_idx = 500 # let's look at trial 501
sample_eeg = np.squeeze(x_train[trial_idx,:,:])
plt.plot(sample_eeg.T)
plt.show()

### Inspect the label information
* It is common to label each example with a vector of probabilities
* Below, the first element of each vector denotes P(non-target)
* Notice above that all but one of the first 10 examples was a non-target. 
    * The 7th example was a target.

In [4]:
print(y_train[0:10,:])

[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]


### Creating the model
* We use the user-friendly Keras library to create a simple model for predicting target vs non-target based on the array of EEG values
* The first layer of our model corresponds to the input data
* The second layer performs a convolution (in effect spatial filtering) across the data in windows of 10 samples each
* The final layer transforms the output of the convolution into a probability value
* Our model has 881 parameters. We have 1058 examples. 
    * It's a good rule-of-thumb to have at least as many training examples as model parameters. 

In [7]:
import keras
input_shape=(64,128,1) # this tells Keras the shape of the data that will be passed to it for training/testing
num_classes=2
num_filters = 1
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        keras.layers.Conv2D(num_filters, kernel_size=(64, 10), activation="relu"),
        keras.layers.Flatten(),
        keras.layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 1, 119, 1)         641       
                                                                 
 flatten (Flatten)           (None, 119)               0         
                                                                 
 dense (Dense)               (None, 2)                 240       
                                                                 
Total params: 881
Trainable params: 881
Non-trainable params: 0
_________________________________________________________________


### Compile the model, 
* We need to tell Keras what type of loss function we will be using, and which optimizer we would like. 
* The batch size controls how many examples are used for each computation of the gradient of the loss function
* The number of epochs controls how many complete passes of the data we would like to make
* We also tell Keras that we want to monitor both the model accuracy, as well as the area under the ROC curve.

In [8]:
batch_size = 128  # how many training examples in each "batch" used to compute gradient of loss function
epochs = 10 # how many passes through the data we want the fitting to take

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy",keras.metrics.AUC()])

### Fitting the model
* Keras allows you to monitor the training performance during model fitting
* Notice that the training performance is quite good
    * Area under the ROC curve of is ~ 0.87
    * Now let's see how well the model generalizes to unseen data

In [11]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)

Epoch 1/10
9/9 [==============================] - 0s 15ms/step - loss: 0.3690 - accuracy: 0.8544 - auc: 0.9202
Epoch 2/10
9/9 [==============================] - 0s 15ms/step - loss: 0.3558 - accuracy: 0.8629 - auc: 0.9261
Epoch 3/10
9/9 [==============================] - 0s 17ms/step - loss: 0.3512 - accuracy: 0.8478 - auc: 0.9275
Epoch 4/10
9/9 [==============================] - 0s 15ms/step - loss: 0.3318 - accuracy: 0.8620 - auc: 0.9338
Epoch 5/10
9/9 [==============================] - 0s 15ms/step - loss: 0.3194 - accuracy: 0.8733 - auc: 0.9396
Epoch 6/10
9/9 [==============================] - 0s 16ms/step - loss: 0.3094 - accuracy: 0.8781 - auc: 0.9432
Epoch 7/10
9/9 [==============================] - 0s 16ms/step - loss: 0.3011 - accuracy: 0.8771 - auc: 0.9457
Epoch 8/10
9/9 [==============================] - 0s 15ms/step - loss: 0.2863 - accuracy: 0.8837 - auc: 0.9506
Epoch 9/10
9/9 [==============================] - 0s 16ms/step - loss: 0.2795 - accuracy: 0.8932 - auc: 0.9527
E

### Evaluate the model on unseen data.
* We use the test set to assess our model performance

In [12]:
metrics = model.evaluate(x_test,y_test)
print("The model accuracy is " + str(metrics[1])) # element 0 is loss, 1 is accuracy, 2 is auroc
print("The model AUROC is " + str(metrics[2]))

17/17 [==============================] - 0s 2ms/step - loss: 0.4401 - accuracy: 0.8377 - auc: 0.9003
The model accuracy is 0.8377358317375183
The model AUROC is 0.9002670049667358


### Verify performance manually

In [ ]:
test_scores = model.predict(x_test)
pred_labels = np.argmax(test_scores,1)
print(test_scores[0:10,:])
print(pred_labels[0:10])

#### In the first array above, the first column is P(non-target), while the second is P(target). In the second array, the binary value indicates which class was predicted (0 for predicted non-target, 1 for predicted target).

In [ ]:
true_labels = np.argmax(y_test,1)
test_accuracy = np.mean(true_labels==pred_labels)
print(test_accuracy)

### Let's try increasing the number of convolution filters

In [ ]:
num_filters = 3
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        keras.layers.Conv2D(num_filters, kernel_size=(64, 10), activation="relu"),
        keras.layers.Flatten(),
        keras.layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

#### Notice that we now have 3 times as many parameters. This puts us at risk of overfitting, but we proceed.

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy",keras.metrics.AUC()])
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)

#### As expected, the training performance gets better. But we need to evaluate the model on _unseen_ data:

In [ ]:
metrics = model.evaluate(x_test,y_test)
print("The model accuracy is " + str(metrics[1]))
print("The model AUROC is " + str(metrics[2]))

#### Despite the number of parameters exceeding the number of training examples, the performance does get better.

### Let's try to improve performance by adding regularization

In [ ]:
reg_l1_l2 = keras.regularizers.l1_l2(l1=0.001, l2=0.001)

model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        keras.layers.Conv2D(num_filters, kernel_size=(64, 10), activation="relu"),
        keras.layers.Flatten(),
        keras.layers.Dense(num_classes, activation="softmax", kernel_regularizer = reg_l1_l2),
    ]
)

model.summary()

#### We have added both L1 and L2 regularization to the dense layer.

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy",keras.metrics.AUC()])
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)
metrics = model.evaluate(x_test,y_test)
print("The model accuracy is " + str(metrics[1]))
print("The model AUROC is " + str(metrics[2]))

#### Regularization may or may not increase the performance in this case (depends on your random seed), but now you know how to implement it in Keras ;-)

### Somewhat advanced: handle unequal class prevalence in the data. Note that targets are infrequent in P300 experiments (by design).

In [ ]:
train_labels = np.argmax(y_train,1)
n_non_targets = np.sum(train_labels==0)
n_targets=np.sum(train_labels==1)
class_ratio = n_targets/n_non_targets
print("The ratio of targets to non targets is " + str(class_ratio))

In [ ]:
class_weight={0:1,1:1/class_ratio}

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy",keras.metrics.AUC()])
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, class_weight=class_weight)
metrics = model.evaluate(x_test,y_test)
print("The model accuracy is " + str(metrics[1]))
print("The model AUROC is " + str(metrics[2]))